## C-GAN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import os
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import time

# Check if CUDA is available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Hyperparameters
latent_dim = 100
image_size = 256  # Image size
batch_size = 32  # Batch size
num_epochs = 200
learning_rate = 0.0002
beta1 = 0.5

# Image transformation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Ensure all images are of the same size
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Load dataset
data_dir = '/raid/ee-mariyam/maryam/ayush/Compressed_Onion_data'  # Update this to your dataset path
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Filter out only onion images
onion_classes = [cls for cls in dataset.classes]
onion_indices = [i for i, (img, label) in enumerate(dataset)]
onion_dataset = Subset(dataset, onion_indices)

# Determine the actual number of onion classes
num_classes = len(onion_classes)
class_to_idx = {cls: idx for idx, cls in enumerate(onion_classes)}

# Verify onion classes and indices
print(f"Onion Classes: {onion_classes}")
print(f"Number of Onion Images: {len(onion_indices)}")
print(f"Number of Onion Classes: {num_classes}")

dataloader = DataLoader(onion_dataset, batch_size=batch_size, shuffle=True)


Onion Classes: ['Onion_Bulb_Rot', 'Onion_IYSV', 'Onion_Thrips']
Number of Onion Images: 2017
Number of Onion Classes: 3


In [2]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LayerNorm(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 2048),
            nn.LayerNorm(2048),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(2048, image_size * image_size * 3),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_input = self.label_emb(labels)
        gen_input = torch.cat((noise, label_input), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), 3, image_size, image_size)
        return img

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.model = nn.Sequential(
            nn.Linear(num_classes + 3 * image_size * image_size, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img, labels):
        label_input = self.label_emb(labels)
        img_flat = img.view(img.size(0), -1)
        d_in = torch.cat((img_flat, label_input), -1)
        validity = self.model(d_in)
        return validity

# Initialize generator and discriminator
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Weight initialization
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)

generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Loss and optimizer
adversarial_loss = nn.BCELoss().to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(beta1, 0.999))

# Training function
def train(generator, discriminator, dataloader, num_epochs):
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for i, (imgs, labels) in enumerate(dataloader):
            batch_size = imgs.size(0)
            
            # Convert labels to match onion class indices
            labels = torch.tensor([class_to_idx[dataset.classes[label]] for label in labels]).to(device)
            if labels.max() >= num_classes:
                raise ValueError(f"  Label out of range: {labels.max()} (num_classes: {num_classes})")

            # Adversarial ground truths
            valid = torch.ones(batch_size, 1).to(device)
            fake = torch.zeros(batch_size, 1).to(device)

            # Configure input
            real_imgs = imgs.to(device)

            # ---------------------
            #  Train Generator
            # ---------------------

            optimizer_G.zero_grad()

            # Sample noise and labels as generator input
            z = torch.randn(batch_size, latent_dim).to(device)
            gen_labels = torch.randint(0, num_classes, (batch_size,)).to(device)
            
            # Generate a batch of images
            gen_imgs = generator(z, gen_labels)

            # Loss measures generator's ability to fool the discriminator
            validity = discriminator(gen_imgs, gen_labels)
            g_loss = adversarial_loss(validity, valid)

            g_loss.backward()
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Loss for real images
            validity_real = discriminator(real_imgs, labels)
            d_real_loss = adversarial_loss(validity_real, valid)

            # Loss for fake images
            validity_fake = discriminator(gen_imgs.detach(), gen_labels)
            d_fake_loss = adversarial_loss(validity_fake, fake)

            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2

            d_loss.backward()
            optimizer_D.step()

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Training completed in {total_time:.2f} seconds")

# Start training
try:
    train(generator, discriminator, dataloader, num_epochs)
except Exception as e:
    print(f"Error during training: {e}")

# Function to denormalize images
def denormalize(tensor):
    return tensor * 0.5 + 0.5  # Rescale to [0, 1]

# Function to generate and save images
def generate_and_save_images(generator, num_images, output_dir, latent_dim, num_classes, image_size, class_names):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate images
    z = torch.randn(num_images, latent_dim).to(device)
    gen_labels = torch.randint(0, num_classes, (num_images,)).to(device)
    gen_imgs = generator(z, gen_labels)
    
    # Denormalize images
    gen_imgs = denormalize(gen_imgs)
    
    # Save images in separate folders
    for i in range(num_images):
        class_name = class_names[gen_labels[i].item()]
        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        save_image(gen_imgs[i].cpu(), os.path.join(class_dir, f"{class_name}_{i}.png"))


Epoch 0/200
Epoch 1/200
Epoch 2/200
Epoch 3/200


KeyboardInterrupt: 

In [ ]:
# Save the generator and discriminator models
torch.save(generator.state_dict(), '/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_CGAN_256_generator.pth')
torch.save(discriminator.state_dict(), '/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_CGAN_256_discriminator.pth')


In [ ]:
# Parameters
num_images_to_generate = 50
output_directory = '/raid/ee-mariyam/maryam/ayush/TiH_Onion_gen_64'

# Generate and save images
generate_and_save_images(generator, num_images_to_generate, output_directory, latent_dim, num_classes, image_size, onion_classes)

## DC-GAN


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import os
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import time

# Check if CUDA is available
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
latent_dim = 100
image_size = 128  # Image size
batch_size = 64  # Batch size
num_epochs = 50
learning_rate = 0.0002
beta1 = 0.5

# Image transformation
transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),  # Ensure all images are of the same size
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]
])

# Load dataset
data_dir = '/raid/ee-mariyam/maryam/ayush/TiH_Onion_data'  # Update this to your dataset path
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Filter out only onion images
onion_classes = [cls for cls in dataset.classes]
onion_indices = [i for i, (img, label) in enumerate(dataset)]
onion_dataset = Subset(dataset, onion_indices)

# Determine the actual number of onion classes
num_classes = len(onion_classes)
class_to_idx = {cls: idx for idx, cls in enumerate(onion_classes)}

# Verify onion classes and indices
print(f"Onion Classes: {onion_classes}")
print(f"Number of Onion Images: {len(onion_indices)}")
print(f"Number of Onion Classes: {num_classes}")

dataloader = DataLoader(onion_dataset, batch_size=batch_size, shuffle=True)

Onion Classes: ['Onion_Bulb_Rot', 'Onion_IYSV', 'Onion_Thrips']
Number of Onion Images: 2017
Number of Onion Classes: 3


In [19]:
num_epochs=50

In [20]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, num_classes)
        self.init_size = image_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim + num_classes, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        label_input = self.label_emb(labels)
        gen_input = torch.cat((noise, label_input), -1)
        out = self.l1(gen_input)
        out = out.view(out.size(0), 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.label_embedding = nn.Embedding(num_classes, num_classes)

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                     nn.LeakyReLU(0.2, inplace=True),
                     nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(3 + num_classes, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = image_size // 2**4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size * ds_size, 1), nn.Sigmoid())

    def forward(self, img, labels):
        label_input = self.label_embedding(labels).view(labels.size(0), num_classes, 1, 1)
        label_input = label_input.repeat(1, 1, img.size(2), img.size(3))
        d_in = torch.cat((img, label_input), 1)
        out = self.model(d_in)
        out = out.view(out.size(0), -1)
        validity = self.adv_layer(out)
        return validity

# Initialize generator and discriminator
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Weight initialization
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)

generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Loss and optimizer
adversarial_loss = nn.BCELoss().to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate, betas=(beta1, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(beta1, 0.999))

# Function to count the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Print the number of parameters in each model
print(f"Number of parameters in Generator: {count_parameters(generator)}")
print(f"Number of parameters in Discriminator: {count_parameters(discriminator)}")

# Training function
def train(generator, discriminator, dataloader, num_epochs):
    start_time = time.time()
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for i, (imgs, labels) in enumerate(dataloader):
            batch_size = imgs.size(0)
            
            # Convert labels to match onion class indices
            labels = torch.tensor([class_to_idx[dataset.classes[label]] for label in labels]).to(device)
            if labels.max() >= num_classes:
                raise ValueError(f"  Label out of range: {labels.max()} (num_classes: {num_classes})")

            # Adversarial ground truths
            valid = torch.ones(batch_size, 1).to(device)
            fake = torch.zeros(batch_size, 1).to(device)

            # Configure input
            real_imgs = imgs.to(device)

            # ---------------------
            #  Train Generator
            # ---------------------

            optimizer_G.zero_grad()

            # Sample noise and labels as generator input
            z = torch.randn(batch_size, latent_dim).to(device)
            gen_labels = torch.randint(0, num_classes, (batch_size,)).to(device)
            
            # Generate a batch of images
            gen_imgs = generator(z, gen_labels)

            # Loss measures generator's ability to fool the discriminator
            validity = discriminator(gen_imgs, gen_labels)
            g_loss = adversarial_loss(validity, valid)

            g_loss.backward()
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Loss for real images
            validity_real = discriminator(real_imgs, labels)
            d_real_loss = adversarial_loss(validity_real, valid)

            # Loss for fake images
            validity_fake = discriminator(gen_imgs.detach(), gen_labels)
            d_fake_loss = adversarial_loss(validity_fake, fake)

            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2

            d_loss.backward()
            optimizer_D.step()

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Training completed in {total_time:.2f} seconds")

# Start training
try:
    train(generator, discriminator, dataloader, num_epochs)
except Exception as e:
    print(f"Error during training: {e}")

# Function to denormalize images
def denormalize(tensor):
    return tensor * 0.5 + 0.5  # Rescale to [0, 1]

# Function to generate and save images
def generate_and_save_images(generator, num_images, output_dir, latent_dim, num_classes, image_size, class_names):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Generate images
    z = torch.randn(num_images, latent_dim).to(device)
    gen_labels = torch.randint(0, num_classes, (num_images,)).to(device)
    gen_imgs = generator(z, gen_labels)
    
    # Denormalize images
    gen_imgs = denormalize(gen_imgs)
    
    # Save images in separate folders
    for i in range(num_images):
        class_name = class_names[gen_labels[i].item()]
        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        save_image(gen_imgs[i].cpu(), os.path.join(class_dir, f"{class_name}_{i}.png"))

Number of parameters in Generator: 13855244
Number of parameters in Discriminator: 106522
Epoch 0/50
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33/50
Epoch 34/50
Epoch 35/50
Epoch 36/50
Epoch 37/50
Epoch 38/50
Epoch 39/50
Epoch 40/50
Epoch 41/50
Epoch 42/50
Epoch 43/50
Epoch 44/50
Epoch 45/50
Epoch 46/50
Epoch 47/50
Epoch 48/50
Epoch 49/50
Training completed in 18311.59 seconds


In [21]:
# Parameters
num_images_to_generate = 500
output_directory = '/raid/ee-mariyam/maryam/ayush/TiH_Onion_gen_DCGAN_128'

# Generate and save images
generate_and_save_images(generator, num_images_to_generate, output_directory, latent_dim, num_classes, image_size, onion_classes)

In [22]:
# Save the generator and discriminator models
torch.save(generator.state_dict(), '/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_DCGAN_128_generator.pth')
torch.save(discriminator.state_dict(), '/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_DCGAN_128_discriminator.pth')


In [23]:
# Create new instances of the Generator and Discriminator classes
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Load the saved state dicts into the models
generator.load_state_dict(torch.load('/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_DCGAN_128_generator.pth'))
discriminator.load_state_dict(torch.load('/raid/ee-mariyam/maryam/ayush/models/TiH_Onion_DCGAN_128_discriminator.pth'))


<All keys matched successfully>

In [25]:
!pip install pytorch-fid

In [27]:
from pytorch_fid import fid_score
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch

# Specify device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize to the input size of Inception-v3
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
])

# Load real and generated datasets
real_dataset = ImageFolder('/raid/ee-mariyam/maryam/ayush/TiH_Onion_data', transform=transform)
generated_dataset = ImageFolder('/raid/ee-mariyam/maryam/ayush/TiH_Onion_gen_DCGAN_128', transform=transform)

# Define function to create a dataloader from the ImageFolder
def create_dataloader(dataset, batch_size):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)

# Create dataloaders
real_dataloader = create_dataloader(real_dataset, batch_size=32)
generated_dataloader = create_dataloader(generated_dataset, batch_size=32)

# Compute the FID score
fid_value = fid_score.calculate_fid_given_paths(
    [real_dataloader, generated_dataloader], 
    device=device, 
    dims=2048
)

print(f"FID Score: {fid_value}")


TypeError: calculate_fid_given_paths() missing 1 required positional argument: 'batch_size'